<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/rnaseq_gbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install scanpy anndata pandas numpy scipy matplotlib statsmodels gseapy openpyxl GEOparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.3/605.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.1/284.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 29.5 MB/s eta 0:00:00


In [ ]:
# ===== Gene sets for preliminary grant analysis =====

STEMNESS = ["SOX2","OLIG2","NES","PROM1","POU3F2","ASCL1","ITGA6","LGR5"]

CIRCADIAN = ["ARNTL","CLOCK","NPAS2","PER1","PER2","PER3","CRY1","CRY2",
             "NR1D1","NR1D2","RORA","DBP","TEF","HLF","CIART"]

METABOLIC_SIMPLE = ["SLC2A1","HK2","PFKM","ALDOA","GAPDH","ENO1","PKM","LDHA",
                    "PDHA1","CS","IDH3A","SDHA","UQCRC1","COX5A","ATP5F1A"]

# senescence: “単一遺伝子で勝負しない”用のcore
SENESCENCE_CORE = ["CDKN1A","CDKN2A","CDKN1B","TP53","GADD45A","GADD45B",
                   "ATM","ATR","CHEK1","CHEK2","SERPINE1","GLB1","IGFBP7"]

# SASP（GBMで刺さる）
SASP = ["IL6","IL8","CXCL1","CXCL2","CCL2","CCL20","MMP1","MMP3","MMP9",
        "ICAM1","PTGS2","TNFAIP3","TGFB1","SERPINE1","PLAUR","IL1B"]

# 増殖（老化/停止と反相関で見せる）
PROLIFERATION = ["MKI67","TOP2A","PCNA","MCM2","MCM3","MCM4","MCM5",
                 "TYMS","CCNB1","CCNB2","CDK1","AURKB"]


In [ ]:
import os, subprocess
from pathlib import Path

GSE = "GSE162931"
BASE = Path("data")/GSE
BASE.mkdir(parents=True, exist_ok=True)

def geo_series_dir(gse: str) -> str:
    # GSE162931 -> series/GSE162nnn/GSE162931
    n = int(gse.replace("GSE",""))
    prefix = f"GSE{n//1000}"
    return f"https://ftp.ncbi.nlm.nih.gov/geo/series/{prefix}nnn/{gse}/suppl"

suppl = geo_series_dir(GSE)

raw_url = f"{suppl}/{GSE}_RAW.tar"
xlsx_urls = [
    f"{suppl}/{GSE}_022timecourse.xlsx",
    f"{suppl}/{GSE}_131timecourse.xlsx",
    f"{suppl}/{GSE}_827timecourse.xlsx",
]

def wget(url, outdir):
    r = subprocess.run(["wget","-nc","-P",str(outdir),url], capture_output=True, text=True)
    if r.returncode != 0:
        print("wget failed:", url)
        print("STDERR:\n", r.stderr[:1000])
        raise RuntimeError(f"wget failed with code {r.returncode}")
    return r

print("Suppl dir:", suppl)

wget(raw_url, BASE)
for u in xlsx_urls:
    wget(u, BASE)

print("Done. Files in folder:")
!ls -lh data/GSE162931


Suppl dir: https://ftp.ncbi.nlm.nih.gov/geo/series/GSE162nnn/GSE162931/suppl
Done. Files in folder:
total 722M
-rw-r--r-- 1 root root  14M Dec  3  2020 GSE162931_022timecourse.xlsx
-rw-r--r-- 1 root root  40M Dec  3  2020 GSE162931_131timecourse.xlsx
-rw-r--r-- 1 root root  12M Dec  3  2020 GSE162931_827timecourse.xlsx
-rw-r--r-- 1 root root 658M Jul 17  2023 GSE162931_RAW.tar


In [ ]:
import os, subprocess, tarfile, glob, shutil
from pathlib import Path
import scanpy as sc
import pandas as pd
import numpy as np

# ===== 基本設定 =====
GSE = "GSE162931"
BASE = Path("data") / GSE
BASE.mkdir(parents=True, exist_ok=True)

# 正しい GEO FTP パスを作る関数
def geo_series_dir(gse: str) -> str:
    n = int(gse.replace("GSE",""))
    prefix = f"GSE{n//1000}"
    return f"https://ftp.ncbi.nlm.nih.gov/geo/series/{prefix}nnn/{gse}/suppl"

suppl = geo_series_dir(GSE)

# ===== 1) ダウンロード =====
raw_tar = BASE / f"{GSE}_RAW.tar"
if not raw_tar.exists():
    subprocess.run(
        ["wget","-nc","-P",str(BASE), f"{suppl}/{GSE}_RAW.tar"],
        check=True
    )

# ===== 2) 展開（★ extract_dir をここで定義）=====
extract_dir = BASE / "RAW_extracted"
extract_dir.mkdir(exist_ok=True)

# まだ展開されていなければ展開
if not any(extract_dir.iterdir()):
    with tarfile.open(raw_tar, "r") as t:
        t.extractall(extract_dir)

print("RAW extracted files:", len(list(extract_dir.glob("*"))))


/tmp/ipython-input-1558456265.py:35: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(extract_dir)


RAW extracted files: 8


In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import shutil

raw = extract_dir
tmp_root = BASE/"tmp_10x"
tmp_root.mkdir(exist_ok=True)

def find_triplets_for_gsm(gsm):
    # GSM4967236_*matrix.mtx.gz / *barcodes.tsv.gz / *features.tsv.gz を探す
    mtx = glob.glob(str(raw/f"{gsm}*matrix.mtx.gz")) + glob.glob(str(raw/f"{gsm}*matrix.mtx"))
    bar = glob.glob(str(raw/f"{gsm}*barcodes.tsv.gz")) + glob.glob(str(raw/f"{gsm}*barcodes.tsv"))
    feat = glob.glob(str(raw/f"{gsm}*features.tsv.gz")) + glob.glob(str(raw/f"{gsm}*features.tsv")) \
         + glob.glob(str(raw/f"{gsm}*genes.tsv.gz")) + glob.glob(str(raw/f"{gsm}*genes.tsv"))
    return (mtx, bar, feat)

def read_gsm_10x(gsm):
    mtx, bar, feat = find_triplets_for_gsm(gsm)
    if len(mtx)==0 or len(bar)==0 or len(feat)==0:
        raise FileNotFoundError(f"{gsm}: triplets not found. mtx={len(mtx)}, bar={len(bar)}, feat={len(feat)}")

    # 一時フォルダを作って標準名にコピー
    td = tmp_root/gsm
    if td.exists():
        shutil.rmtree(td)
    td.mkdir()

    shutil.copy(mtx[0], td/"matrix.mtx.gz" if mtx[0].endswith(".gz") else td/"matrix.mtx")
    shutil.copy(bar[0], td/"barcodes.tsv.gz" if bar[0].endswith(".gz") else td/"barcodes.tsv")
    # features.tsv or genes.tsv → features.tsv(.gz)に
    shutil.copy(feat[0], td/"features.tsv.gz" if feat[0].endswith(".gz") else td/"features.tsv")

    ad = sc.read_10x_mtx(str(td), var_names="gene_symbols", cache=False)
    ad.var_names_make_unique()
    ad.obs["GSM"] = gsm
    return ad

# GSE162931はGEOページに8サンプル（タイトル）として列挙されています :contentReference[oaicite:2]{index=2}
GSM_LIST = [
    "GSM4967236","GSM4967237","GSM4967239","GSM4967241",
    "GSM4967242","GSM4967244","GSM4967246","GSM4967248"
]

adatas = []
for gsm in GSM_LIST:
    print("Reading", gsm)
    adatas.append(read_gsm_10x(gsm))

adata = sc.concat(adatas, join="outer", label="GSM", keys=[a.obs["GSM"][0] for a in adatas])
print(adata)


NameError: name 'extract_dir' is not defined

In [ ]:
# GEOページのサンプルタイトルに基づく（naive / RT 2 days / RT 5 days / RT 3 weeks） :contentReference[oaicite:4]{index=4}
meta = {
    "GSM4967236": dict(line="GBM131", condition="naive", timepoint="0d"),
    "GSM4967237": dict(line="GBM131", condition="RT",    timepoint="2d"),
    "GSM4967239": dict(line="GBM131", condition="RT",    timepoint="5d"),
    "GSM4967241": dict(line="GBM131", condition="RT",    timepoint="3w"),
    "GSM4967242": dict(line="GBM022", condition="naive", timepoint="0d"),
    "GSM4967244": dict(line="GBM022", condition="RT",    timepoint="2d"),
    "GSM4967246": dict(line="GBM827", condition="naive", timepoint="0d"),
    "GSM4967248": dict(line="GBM827", condition="RT",    timepoint="2d"),
}

adata.obs["line"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["line"])
adata.obs["condition"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["condition"])
adata.obs["timepoint"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["timepoint"])

adata.obs[["GSM","line","condition","timepoint"]].head()


In [ ]:
import matplotlib.pyplot as plt

OUTDIR = Path("out_GSE162931")
OUTDIR.mkdir(exist_ok=True)
sc.settings.figdir = str(OUTDIR)
sc.settings.set_figure_params(dpi=220, fontsize=10)

# QC
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=10)

adata.var["mt"] = adata.var_names.str.upper().str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
adata = adata[adata.obs.pct_counts_mt < 20].copy()

# Normalize / HVG / PCA / UMAP
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000, subset=True)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.6)

def score(genes, name):
    g = [x for x in genes if x in adata.var_names]
    if len(g) < 4:
        adata.obs[name] = np.nan
        print(f"[WARN] {name}: found {len(g)} genes")
        return
    sc.tl.score_genes(adata, gene_list=g, score_name=name)

# 4本柱 + proliferation
score(STEMNESS, "StemnessScore")
score(METABOLIC_SIMPLE, "MetabolicScore")
score(CIRCADIAN, "CircadianScore")
score(SENESCENCE_CORE, "SenescenceCoreScore")
score(SASP, "SASPScore")
score(PROLIFERATION, "ProliferationScore")

# 1) UMAP (meta)
sc.pl.umap(adata, color=["line","condition","timepoint","leiden"], wspace=0.4,
           save="_UMAP_meta.png", show=False)

# 2) UMAP (scores)
sc.pl.umap(adata, color=[
    "StemnessScore","MetabolicScore","CircadianScore","SenescenceCoreScore","SASPScore","ProliferationScore"
], wspace=0.4, save="_UMAP_scores.png", show=False)

# 3) Violin by condition (naive vs RT)
sc.pl.violin(adata,
             keys=["SenescenceCoreScore","SASPScore","StemnessScore","CircadianScore","MetabolicScore","ProliferationScore"],
             groupby="condition", rotation=45,
             save="_Scores_byCondition.png", show=False)

# 4) Violin by timepoint（time-courseが一枚で出る）
sc.pl.violin(adata,
             keys=["SenescenceCoreScore","SASPScore","StemnessScore","CircadianScore","MetabolicScore","ProliferationScore"],
             groupby="timepoint", rotation=45,
             save="_Scores_byTimepoint.png", show=False)

# 5) クラスター比率（治療後に増える集団）
ct = pd.crosstab(adata.obs["leiden"], adata.obs["condition"], normalize="columns")
ct.to_csv(OUTDIR/"cluster_fraction_by_condition.csv")

# 6) DEG（condition）
sc.tl.rank_genes_groups(adata, groupby="condition", method="wilcoxon")
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, save="_DEG_byCondition.png", show=False)

# 保存
adata.write(BASE/f"{GSE}.h5ad")
print("Saved:", BASE/f"{GSE}.h5ad")
print("Figures in:", OUTDIR)


In [ ]:
import os, subprocess, tarfile
from pathlib import Path

GSE = "GSE162931"
BASE = Path("data")/GSE
BASE.mkdir(parents=True, exist_ok=True)

def geo_suppl_url(gse: str) -> str:
    n = int(gse.replace("GSE",""))
    prefix = f"GSE{n//1000}"   # 例: 162931 -> GSE162
    return f"https://ftp.ncbi.nlm.nih.gov/geo/series/{prefix}nnn/{gse}/suppl"

suppl = geo_suppl_url(GSE)
tar_path = BASE/f"{GSE}_RAW.tar"

# 1) download RAW.tar
if not tar_path.exists():
    subprocess.run(["wget","-nc","-P",str(BASE), f"{suppl}/{GSE}_RAW.tar"], check=True)

# 2) extract to RAW_extracted
extract_dir = BASE/"RAW_extracted"
extract_dir.mkdir(exist_ok=True)

if not any(extract_dir.iterdir()):
    with tarfile.open(tar_path, "r") as t:
        t.extractall(extract_dir)

print("OK: extracted")
print("Folder:", extract_dir)
print("---- first 100 files ----")
files = sorted([p.name for p in extract_dir.iterdir()])
print("\n".join(files[:100]))
print("---- total files ----", len(files))


OK: extracted
Folder: data/GSE162931/RAW_extracted
---- first 100 files ----
GSM4967236_Lee1_filtered_feature_bc_matrix.tar.gz
GSM4967237_Lee2_filtered_feature_bc_matrix.tar.gz
GSM4967239_Lee3_filtered_feature_bc_matrix.tar.gz
GSM4967241_Lee4_filtered_feature_bc_matrix.tar.gz
GSM4967242_Lee5_filtered_feature_bc_matrix.tar.gz
GSM4967244_Lee6_filtered_feature_bc_matrix.tar.gz
GSM4967246_Lee7_filtered_feature_bc_matrix.tar.gz
GSM4967248_Lee8_filtered_feature_bc_matrix.tar.gz
---- total files ---- 8


In [ ]:
import re, glob, shutil
from pathlib import Path
import scanpy as sc

# セル1で作られた変数を使う（GSE, BASE, extract_dir）
tmp_root = BASE/"tmp_10x"
tmp_root.mkdir(exist_ok=True)

# 1) extract_dir内から "GSMxxxxxxx" をすべて拾う
all_names = [p.name for p in extract_dir.iterdir()]
gsm_set = sorted(set(re.findall(r"(GSM\d+)", "\n".join(all_names))))
print("Detected GSMs:", len(gsm_set))
print("Example GSMs:", gsm_set[:10])

def pick_one(patterns):
    for pat in patterns:
        hits = glob.glob(str(extract_dir/pat))
        if hits:
            return hits[0]
    return None

def read_one_gsm(gsm):
    # 2) よくある命名揺れに全部対応（matrix / barcodes / features / genes）
    mtx = pick_one([f"{gsm}*matrix.mtx.gz", f"{gsm}*matrix.mtx"])
    bar = pick_one([f"{gsm}*barcodes.tsv.gz", f"{gsm}*barcodes.tsv"])
    feat = pick_one([f"{gsm}*features.tsv.gz", f"{gsm}*features.tsv",
                     f"{gsm}*genes.tsv.gz", f"{gsm}*genes.tsv"])
    if not (mtx and bar and feat):
        raise FileNotFoundError(f"{gsm}: missing triplet -> mtx={bool(mtx)}, bar={bool(bar)}, feat={bool(feat)}")

    # 3) Scanpyが読める標準名で一時フォルダへ
    td = tmp_root/gsm
    if td.exists(): shutil.rmtree(td)
    td.mkdir()

    shutil.copy(mtx, td/("matrix.mtx.gz" if str(mtx).endswith(".gz") else "matrix.mtx"))
    shutil.copy(bar, td/("barcodes.tsv.gz" if str(bar).endswith(".gz") else "barcodes.tsv"))
    shutil.copy(feat, td/("features.tsv.gz" if str(feat).endswith(".gz") else "features.tsv"))

    ad = sc.read_10x_mtx(str(td), var_names="gene_symbols", cache=False)
    ad.var_names_make_unique()
    ad.obs["GSM"] = gsm
    return ad

# 4) 全GSMを読んで結合（うまくいかないGSMがあっても止めずに進める）
adatas = []
failed = []
for gsm in gsm_set:
    try:
        print("Reading", gsm)
        adatas.append(read_one_gsm(gsm))
    except Exception as e:
        failed.append((gsm, str(e)))

print("Read OK:", len(adatas))
print("Failed:", len(failed))
if failed:
    print("---- failed list (first 10) ----")
    for x in failed[:10]:
        print(x)

if len(adatas) == 0:
    raise RuntimeError("No samples could be read. Show the file list from cell1 output.")

adata = sc.concat(adatas, join="outer")
print(adata)

# 5) とりあえず h5ad に保存（ここまで来れば勝ち）
out_h5ad = BASE/f"{GSE}_auto.h5ad"
adata.write(out_h5ad)
print("Saved:", out_h5ad)


Detected GSMs: 8
Example GSMs: ['GSM4967236', 'GSM4967237', 'GSM4967239', 'GSM4967241', 'GSM4967242', 'GSM4967244', 'GSM4967246', 'GSM4967248']
Reading GSM4967236
Reading GSM4967237
Reading GSM4967239
Reading GSM4967241
Reading GSM4967242
Reading GSM4967244
Reading GSM4967246
Reading GSM4967248
Read OK: 0
Failed: 8
---- failed list (first 10) ----
('GSM4967236', 'GSM4967236: missing triplet -> mtx=False, bar=False, feat=False')
('GSM4967237', 'GSM4967237: missing triplet -> mtx=False, bar=False, feat=False')
('GSM4967239', 'GSM4967239: missing triplet -> mtx=False, bar=False, feat=False')
('GSM4967241', 'GSM4967241: missing triplet -> mtx=False, bar=False, feat=False')
('GSM4967242', 'GSM4967242: missing triplet -> mtx=False, bar=False, feat=False')
('GSM4967244', 'GSM4967244: missing triplet -> mtx=False, bar=False, feat=False')
('GSM4967246', 'GSM4967246: missing triplet -> mtx=False, bar=False, feat=False')
('GSM4967248', 'GSM4967248: missing triplet -> mtx=False, bar=False, feat=Fa

RuntimeError: No samples could be read. Show the file list from cell1 output.

In [ ]:
from pathlib import Path
import re

# ここはセル1で作ったものを使う想定
# GSE, BASE, extract_dir が無い場合は：
# GSE="GSE162931"; BASE=Path("data")/GSE; extract_dir=BASE/"RAW_extracted"

extract_dir = Path(extract_dir)

def is_10x_dir(d: Path) -> bool:
    names = {p.name for p in d.iterdir() if p.is_file()}
    has_mtx = any(n.endswith("matrix.mtx") or n.endswith("matrix.mtx.gz") for n in names)
    has_bar = any(n.endswith("barcodes.tsv") or n.endswith("barcodes.tsv.gz") for n in names)
    has_feat = any(n.endswith("features.tsv") or n.endswith("features.tsv.gz") or
                   n.endswith("genes.tsv") or n.endswith("genes.tsv.gz") for n in names)
    return has_mtx and has_bar and has_feat

# RAW_extracted配下の全フォルダを走査して、10x三点セットが揃った場所を拾う
cand_dirs = []
for d in extract_dir.rglob("*"):
    if d.is_dir():
        try:
            if is_10x_dir(d):
                cand_dirs.append(d)
        except PermissionError:
            pass

print("10x candidate dirs found:", len(cand_dirs))
for d in cand_dirs[:20]:
    print(" -", d)


10x candidate dirs found: 0


In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np

if len(cand_dirs) == 0:
    raise RuntimeError("10xフォルダが1つも見つかりません。RAW.tarが10x形式じゃない可能性があります。次のセルCへ。")

adatas = []
for d in cand_dirs:
    print("Reading:", d)
    ad = sc.read_10x_mtx(str(d), var_names="gene_symbols", cache=False)
    ad.var_names_make_unique()
    # サンプル名（フォルダ名など）を入れておく
    ad.obs["sample_dir"] = str(d.relative_to(extract_dir))
    # ついでにGSMっぽい文字列がフォルダパスに含まれていれば抽出
    m = re.search(r"(GSM\d+)", str(d))
    ad.obs["GSM_guess"] = m.group(1) if m else "NA"
    adatas.append(ad)

adata = sc.concat(adatas, join="outer")
print(adata)

out_h5ad = Path(BASE)/f"{GSE}_auto.h5ad"
adata.write(out_h5ad)
print("Saved:", out_h5ad)
print("obs columns:", list(adata.obs.columns))
print(adata.obs[["sample_dir","GSM_guess"]].drop_duplicates().head(20))


RuntimeError: 10xフォルダが1つも見つかりません。RAW.tarが10x形式じゃない可能性があります。次のセルCへ。

In [ ]:
import tarfile, re
from collections import Counter
from pathlib import Path

GSE = "GSE162931"
BASE = Path("data")/GSE
tar_path = BASE/f"{GSE}_RAW.tar"

with tarfile.open(tar_path, "r") as t:
    names = t.getnames()

print("Total files in tar:", len(names))

# 拡張子トップ20
exts = []
for n in names:
    m = re.search(r"(\.[A-Za-z0-9]+(?:\.gz)?)$", n)
    exts.append(m.group(1).lower() if m else "(none)")
print("Top extensions:", Counter(exts).most_common(20))

# 10xっぽい単語が含まれるファイルを抽出
keys = ["matrix.mtx", "barcodes.tsv", "features.tsv", "genes.tsv", ".h5", "filtered_feature_bc_matrix", "raw_feature_bc_matrix"]
for k in keys:
    hits = [n for n in names if k in n]
    print(f"\n--- hits for '{k}' ({len(hits)}) ---")
    print("\n".join(hits[:30]))

# ファイル名先頭100件
print("\n--- first 120 file names ---")
print("\n".join(names[:120]))


Total files in tar: 8
Top extensions: [('.tar.gz', 8)]

--- hits for 'matrix.mtx' (0) ---


--- hits for 'barcodes.tsv' (0) ---


--- hits for 'features.tsv' (0) ---


--- hits for 'genes.tsv' (0) ---


--- hits for '.h5' (0) ---


--- hits for 'filtered_feature_bc_matrix' (8) ---
GSM4967236_Lee1_filtered_feature_bc_matrix.tar.gz
GSM4967237_Lee2_filtered_feature_bc_matrix.tar.gz
GSM4967239_Lee3_filtered_feature_bc_matrix.tar.gz
GSM4967241_Lee4_filtered_feature_bc_matrix.tar.gz
GSM4967242_Lee5_filtered_feature_bc_matrix.tar.gz
GSM4967244_Lee6_filtered_feature_bc_matrix.tar.gz
GSM4967246_Lee7_filtered_feature_bc_matrix.tar.gz
GSM4967248_Lee8_filtered_feature_bc_matrix.tar.gz

--- hits for 'raw_feature_bc_matrix' (0) ---


--- first 120 file names ---
GSM4967236_Lee1_filtered_feature_bc_matrix.tar.gz
GSM4967237_Lee2_filtered_feature_bc_matrix.tar.gz
GSM4967239_Lee3_filtered_feature_bc_matrix.tar.gz
GSM4967241_Lee4_filtered_feature_bc_matrix.tar.gz
GSM4967242_Lee5_filtered_feature_bc_matri

In [ ]:
import os, tarfile, re, glob
from pathlib import Path
import scanpy as sc
import pandas as pd
import numpy as np

# ===== settings =====
GSE = "GSE162931"
BASE = Path("data")/GSE
raw_tar = BASE/f"{GSE}_RAW.tar"
inner_dir = BASE/"inner_tarballs"
inner_dir.mkdir(exist_ok=True)
extract_root = BASE/"samples_extracted"
extract_root.mkdir(exist_ok=True)

# ===== 1) outer tar: extract 8 inner tar.gz files =====
with tarfile.open(raw_tar, "r") as t:
    t.extractall(inner_dir)

inner_tars = sorted(inner_dir.glob("GSM*_filtered_feature_bc_matrix.tar.gz"))
print("Inner tar.gz files:", len(inner_tars))
print([p.name for p in inner_tars])

# ===== 2) extract each inner tar.gz to its own folder =====
sample_dirs = []
for tgz in inner_tars:
    # sample name like GSM4967236_Lee1
    m = re.match(r"(GSM\d+)_([^_]+)_filtered_feature_bc_matrix\.tar\.gz", tgz.name)
    gsm = m.group(1) if m else "NA"
    tag = m.group(2) if m else tgz.stem

    out = extract_root/f"{gsm}_{tag}"
    out.mkdir(exist_ok=True)

    # extract only once
    if not any(out.rglob("*matrix.mtx*")):
        with tarfile.open(tgz, "r:gz") as t:
            t.extractall(out)

    # find the actual filtered_feature_bc_matrix directory
    # it can be at out/filtered_feature_bc_matrix or deeper
    candidates = [p for p in out.rglob("filtered_feature_bc_matrix") if p.is_dir()]
    if len(candidates)==0:
        raise FileNotFoundError(f"filtered_feature_bc_matrix dir not found inside {tgz.name}")
    fdir = candidates[0]
    sample_dirs.append((gsm, tag, fdir))

print("Detected 10x dirs:", len(sample_dirs))
for x in sample_dirs:
    print(" -", x[0], x[1], "->", x[2])

# ===== 3) read each sample and concat =====
adatas = []
for gsm, tag, fdir in sample_dirs:
    print("Reading:", gsm, tag)
    ad = sc.read_10x_mtx(str(fdir), var_names="gene_symbols", cache=False)
    ad.var_names_make_unique()
    ad.obs["GSM"] = gsm
    ad.obs["sample_tag"] = tag
    adatas.append(ad)

adata = sc.concat(adatas, join="outer", label="sample", keys=[f"{g}_{t}" for g,t,_ in sample_dirs])
print(adata)

# ===== 4) attach condition/timepoint/line using known mapping (fast & grant-friendly) =====
# GSE162931 series page sample titles correspond to these GSMs:
meta = {
    "GSM4967236": dict(line="GBM131", condition="naive", timepoint="0d"),
    "GSM4967237": dict(line="GBM131", condition="RT",    timepoint="2d"),
    "GSM4967239": dict(line="GBM131", condition="RT",    timepoint="5d"),
    "GSM4967241": dict(line="GBM131", condition="RT",    timepoint="3w"),
    "GSM4967242": dict(line="GBM022", condition="naive", timepoint="0d"),
    "GSM4967244": dict(line="GBM022", condition="RT",    timepoint="2d"),
    "GSM4967246": dict(line="GBM827", condition="naive", timepoint="0d"),
    "GSM4967248": dict(line="GBM827", condition="RT",    timepoint="2d"),
}
adata.obs["line"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["line"])
adata.obs["condition"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["condition"])
adata.obs["timepoint"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["timepoint"])

# ===== 5) save h5ad (base) =====
out_h5ad = BASE/f"{GSE}_merged.h5ad"
adata.write(out_h5ad)
print("Saved:", out_h5ad)

# quick check
print(adata.obs[["GSM","sample_tag","line","condition","timepoint"]].drop_duplicates().sort_values(["line","timepoint"]))


/tmp/ipython-input-1278877599.py:18: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(inner_dir)


Inner tar.gz files: 8
['GSM4967236_Lee1_filtered_feature_bc_matrix.tar.gz', 'GSM4967237_Lee2_filtered_feature_bc_matrix.tar.gz', 'GSM4967239_Lee3_filtered_feature_bc_matrix.tar.gz', 'GSM4967241_Lee4_filtered_feature_bc_matrix.tar.gz', 'GSM4967242_Lee5_filtered_feature_bc_matrix.tar.gz', 'GSM4967244_Lee6_filtered_feature_bc_matrix.tar.gz', 'GSM4967246_Lee7_filtered_feature_bc_matrix.tar.gz', 'GSM4967248_Lee8_filtered_feature_bc_matrix.tar.gz']


/tmp/ipython-input-1278877599.py:38: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(out)


FileNotFoundError: filtered_feature_bc_matrix dir not found inside GSM4967236_Lee1_filtered_feature_bc_matrix.tar.gz

In [ ]:
import tarfile, re
from pathlib import Path
import scanpy as sc
import pandas as pd
import numpy as np

# ===== settings =====
GSE = "GSE162931"
BASE = Path("data")/GSE
raw_tar = BASE/f"{GSE}_RAW.tar"

inner_dir = BASE/"inner_tarballs"
extract_root = BASE/"samples_extracted"
inner_dir.mkdir(exist_ok=True)
extract_root.mkdir(exist_ok=True)

# ===== 1) outer tar: extract inner tar.gz =====
with tarfile.open(raw_tar, "r") as t:
    t.extractall(inner_dir)

inner_tars = sorted(inner_dir.glob("GSM*_filtered_feature_bc_matrix.tar.gz"))
print("Inner tar.gz:", [p.name for p in inner_tars])

# ===== 2) extract each inner tar.gz and READ IT DIRECTLY =====
adatas = []

for tgz in inner_tars:
    print("Processing:", tgz.name)

    m = re.match(r"(GSM\d+)_([^_]+)_filtered_feature_bc_matrix\.tar\.gz", tgz.name)
    gsm = m.group(1)
    tag = m.group(2)

    out = extract_root/f"{gsm}_{tag}"
    out.mkdir(exist_ok=True)

    # extract tar.gz
    if not any(out.iterdir()):
        with tarfile.open(tgz, "r:gz") as t:
            t.extractall(out)

    # ★ここが重要★
    # filtered_feature_bc_matrix フォルダを探さない
    # 展開先 out 自体を 10x ディレクトリとして読む
    print("  Reading 10x from:", out)

    ad = sc.read_10x_mtx(str(out), var_names="gene_symbols", cache=False)
    ad.var_names_make_unique()
    ad.obs["GSM"] = gsm
    ad.obs["sample_tag"] = tag

    adatas.append(ad)

# ===== 3) concat =====
adata = sc.concat(adatas, join="outer", label="sample",
                  keys=[f"{a.obs['GSM'][0]}_{a.obs['sample_tag'][0]}" for a in adatas])

print(adata)

# ===== 4) attach metadata (RT time course) =====
meta = {
    "GSM4967236": dict(line="GBM131", condition="naive", timepoint="0d"),
    "GSM4967237": dict(line="GBM131", condition="RT",    timepoint="2d"),
    "GSM4967239": dict(line="GBM131", condition="RT",    timepoint="5d"),
    "GSM4967241": dict(line="GBM131", condition="RT",    timepoint="3w"),
    "GSM4967242": dict(line="GBM022", condition="naive", timepoint="0d"),
    "GSM4967244": dict(line="GBM022", condition="RT",    timepoint="2d"),
    "GSM4967246": dict(line="GBM827", condition="naive", timepoint="0d"),
    "GSM4967248": dict(line="GBM827", condition="RT",    timepoint="2d"),
}

adata.obs["line"] = adata.obs["GSM"].map(lambda x: meta[x]["line"])
adata.obs["condition"] = adata.obs["GSM"].map(lambda x: meta[x]["condition"])
adata.obs["timepoint"] = adata.obs["GSM"].map(lambda x: meta[x]["timepoint"])

# ===== 5) save =====
out_h5ad = BASE/f"{GSE}_merged.h5ad"
adata.write(out_h5ad)
print("Saved:", out_h5ad)

print(adata.obs[["GSM","sample_tag","line","condition","timepoint"]].drop_duplicates())


/tmp/ipython-input-2219174894.py:19: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(inner_dir)


Inner tar.gz: ['GSM4967236_Lee1_filtered_feature_bc_matrix.tar.gz', 'GSM4967237_Lee2_filtered_feature_bc_matrix.tar.gz', 'GSM4967239_Lee3_filtered_feature_bc_matrix.tar.gz', 'GSM4967241_Lee4_filtered_feature_bc_matrix.tar.gz', 'GSM4967242_Lee5_filtered_feature_bc_matrix.tar.gz', 'GSM4967244_Lee6_filtered_feature_bc_matrix.tar.gz', 'GSM4967246_Lee7_filtered_feature_bc_matrix.tar.gz', 'GSM4967248_Lee8_filtered_feature_bc_matrix.tar.gz']
Processing: GSM4967236_Lee1_filtered_feature_bc_matrix.tar.gz
  Reading 10x from: data/GSE162931/samples_extracted/GSM4967236_Lee1
Processing: GSM4967237_Lee2_filtered_feature_bc_matrix.tar.gz


/tmp/ipython-input-2219174894.py:40: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(out)


  Reading 10x from: data/GSE162931/samples_extracted/GSM4967237_Lee2
Processing: GSM4967239_Lee3_filtered_feature_bc_matrix.tar.gz


/tmp/ipython-input-2219174894.py:40: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(out)


...

  Reading 10x from: data/GSE162931/samples_extracted/GSM4967246_Lee7
Processing: GSM4967248_Lee8_filtered_feature_bc_matrix.tar.gz


/tmp/ipython-input-2219174894.py:40: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(out)


  Reading 10x from: data/GSE162931/samples_extracted/GSM4967248_Lee8


/tmp/ipython-input-2219174894.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  keys=[f"{a.obs['GSM'][0]}_{a.obs['sample_tag'][0]}" for a in adatas])
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 90822 × 33694
    obs: 'GSM', 'sample_tag', 'sample'
Saved: data/GSE162931/GSE162931_merged.h5ad
                           GSM sample_tag    line condition timepoint
AAACCCAAGCAAACAT-1  GSM4967236       Lee1  GBM131     naive        0d
AAACCCAAGATGAACT-1  GSM4967237       Lee2  GBM131        RT        2d
AAACCCAAGACGCCCT-1  GSM4967239       Lee3  GBM131        RT        5d
AAACCCAAGAGCTGCA-1  GSM4967241       Lee4  GBM131        RT        3w
AAACCCAAGACGCCCT-1  GSM4967242       Lee5  GBM022     naive        0d
AAACCCAAGGAGGGTG-1  GSM4967244       Lee6  GBM022        RT        2d
AAACCCAAGGGAGTGG-1  GSM4967246       Lee7  GBM827     naive        0d
AAACCCAAGCCGTCGT-1  GSM4967248       Lee8  GBM827        RT        2d


In [ ]:
import tarfile, re
from pathlib import Path
import scanpy as sc

GSE = "GSE162931"
BASE = Path("data")/GSE
raw_tar = BASE/f"{GSE}_RAW.tar"

inner_dir = BASE/"inner_tarballs"
extract_root = BASE/"samples_extracted"
inner_dir.mkdir(exist_ok=True)
extract_root.mkdir(exist_ok=True)

# 1) outer tar → inner tar.gz を取り出す
with tarfile.open(raw_tar, "r") as t:
    t.extractall(inner_dir)

inner_tars = sorted(inner_dir.glob("GSM*_filtered_feature_bc_matrix.tar.gz"))
print("Inner tar.gz files:", len(inner_tars))

# 2) inner tar.gz を展開して、その展開先(out)を10xとして読む
adatas = []
for tgz in inner_tars:
    m = re.match(r"(GSM\d+)_([^_]+)_filtered_feature_bc_matrix\.tar\.gz", tgz.name)
    gsm = m.group(1); tag = m.group(2)

    out = extract_root/f"{gsm}_{tag}"
    out.mkdir(exist_ok=True)

    if not any(out.iterdir()):
        with tarfile.open(tgz, "r:gz") as t:
            t.extractall(out)

    print("Reading:", gsm, tag, "from", out)
    ad = sc.read_10x_mtx(str(out), var_names="gene_symbols", cache=False)
    ad.var_names_make_unique()
    ad.obs["GSM"] = gsm
    ad.obs["sample_tag"] = tag
    adatas.append(ad)

adata = sc.concat(adatas, join="outer")
print("Merged:", adata)

# 3) 保存（ここまでできれば勝ち）
out_h5ad = BASE/f"{GSE}_merged.h5ad"
adata.write(out_h5ad)
print("Saved:", out_h5ad)


/tmp/ipython-input-940012099.py:16: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(inner_dir)


Inner tar.gz files: 8
Reading: GSM4967236 Lee1 from data/GSE162931/samples_extracted/GSM4967236_Lee1
Reading: GSM4967237 Lee2 from data/GSE162931/samples_extracted/GSM4967237_Lee2
Reading: GSM4967239 Lee3 from data/GSE162931/samples_extracted/GSM4967239_Lee3
Reading: GSM4967241 Lee4 from data/GSE162931/samples_extracted/GSM4967241_Lee4
Reading: GSM4967242 Lee5 from data/GSE162931/samples_extracted/GSM4967242_Lee5
Reading: GSM4967244 Lee6 from data/GSE162931/samples_extracted/GSM4967244_Lee6
Reading: GSM4967246 Lee7 from data/GSE162931/samples_extracted/GSM4967246_Lee7
Reading: GSM4967248 Lee8 from data/GSE162931/samples_extracted/GSM4967248_Lee8


/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Merged: AnnData object with n_obs × n_vars = 90822 × 33694
    obs: 'GSM', 'sample_tag'
Saved: data/GSE162931/GSE162931_merged.h5ad


In [ ]:
STEMNESS = ["SOX2","OLIG2","NES","PROM1","POU3F2","ASCL1","ITGA6","LGR5"]

CIRCADIAN = ["ARNTL","CLOCK","NPAS2","PER1","PER2","PER3","CRY1","CRY2",
             "NR1D1","NR1D2","RORA","DBP","TEF","HLF","CIART"]

METABOLIC_SIMPLE = ["SLC2A1","HK2","PFKM","ALDOA","GAPDH","ENO1","PKM","LDHA",
                    "PDHA1","CS","IDH3A","SDHA","UQCRC1","COX5A","ATP5F1A"]

SENESCENCE_CORE = ["CDKN1A","CDKN2A","CDKN1B","TP53","GADD45A","GADD45B",
                   "ATM","ATR","CHEK1","CHEK2","SERPINE1","GLB1","IGFBP7"]

SASP = ["IL6","IL8","CXCL1","CXCL2","CCL2","CCL20","MMP1","MMP3","MMP9",
        "ICAM1","PTGS2","TNFAIP3","TGFB1","SERPINE1","PLAUR","IL1B"]

PROLIFERATION = ["MKI67","TOP2A","PCNA","MCM2","MCM3","MCM4","MCM5",
                 "TYMS","CCNB1","CCNB2","CDK1","AURKB"]


In [ ]:
!pip install -q igraph leidenalg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 100.2 MB/s eta 0:00:00


In [ ]:
import scanpy as sc
import numpy as np
from pathlib import Path

GSE = "GSE162931"
BASE = Path("data")/GSE
adata = sc.read_h5ad(BASE/f"{GSE}_merged.h5ad")

# ===== メタデータ付与（必要）=====
meta = {
    "GSM4967236": dict(line="GBM131", condition="naive", timepoint="0d"),
    "GSM4967237": dict(line="GBM131", condition="RT",    timepoint="2d"),
    "GSM4967239": dict(line="GBM131", condition="RT",    timepoint="5d"),
    "GSM4967241": dict(line="GBM131", condition="RT",    timepoint="3w"),
    "GSM4967242": dict(line="GBM022", condition="naive", timepoint="0d"),
    "GSM4967244": dict(line="GBM022", condition="RT",    timepoint="2d"),
    "GSM4967246": dict(line="GBM827", condition="naive", timepoint="0d"),
    "GSM4967248": dict(line="GBM827", condition="RT",    timepoint="2d"),
}
adata.obs["line"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["line"])
adata.obs["condition"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["condition"])
adata.obs["timepoint"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["timepoint"])

# ===== 出力先 =====
OUTDIR = Path("out_GSE162931")
OUTDIR.mkdir(exist_ok=True)
sc.settings.figdir = str(OUTDIR)
sc.settings.set_figure_params(dpi=220, fontsize=10)

# ===== 前処理 =====
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=10)
adata.var["mt"] = adata.var_names.str.upper().str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
adata = adata[adata.obs.pct_counts_mt < 20].copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000, subset=True)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.6)

def score(genes, name):
    g = [x for x in genes if x in adata.var_names]
    if len(g) < 4:
        adata.obs[name] = np.nan
        print(f"[WARN] {name}: found {len(g)} genes")
        return
    sc.tl.score_genes(adata, g, score_name=name)

# ===== 4本柱 + SASP + 増殖 =====
score(STEMNESS, "StemnessScore")
score(METABOLIC_SIMPLE, "MetabolicScore")
score(CIRCADIAN, "CircadianScore")
score(SENESCENCE_CORE, "SenescenceCoreScore")
score(SASP, "SASPScore")
score(PROLIFERATION, "ProliferationScore")

# ===== 図（助成金用セット）=====
sc.pl.umap(adata, color=["line","condition","timepoint","leiden"], wspace=0.4, show=False, save="_meta.png")
sc.pl.umap(adata, color=["StemnessScore","MetabolicScore","CircadianScore","SenescenceCoreScore","SASPScore","ProliferationScore"],
           wspace=0.4, show=False, save="_scores.png")

sc.pl.violin(adata,
             keys=["StemnessScore","MetabolicScore","CircadianScore","SenescenceCoreScore","SASPScore","ProliferationScore"],
             groupby="condition", rotation=45, show=False, save="_scores_by_condition.png")

sc.pl.violin(adata,
             keys=["StemnessScore","MetabolicScore","CircadianScore","SenescenceCoreScore","SASPScore","ProliferationScore"],
             groupby="timepoint", rotation=45, show=False, save="_scores_by_timepoint.png")

# DEG（RT vs naive）
sc.tl.rank_genes_groups(adata, groupby="condition", method="wilcoxon")
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, show=False, save="_DEG_condition.png")

# 保存
adata.write(OUTDIR/"GSE162931_processed_scores.h5ad")
print("DONE. Figures in:", OUTDIR)


/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  

In [ ]:
# ===== Gene sets (REDEFINE after runtime crash) =====

STEMNESS = [
    "SOX2","OLIG2","NES","PROM1","POU3F2",
    "ASCL1","ITGA6","LGR5"
]

CIRCADIAN = [
    "ARNTL","CLOCK","NPAS2",
    "PER1","PER2","PER3",
    "CRY1","CRY2",
    "NR1D1","NR1D2",
    "RORA","DBP","TEF","HLF","CIART"
]

METABOLIC_SIMPLE = [
    "SLC2A1","HK2","PFKM","ALDOA","GAPDH","ENO1","PKM","LDHA",
    "PDHA1","CS","IDH3A","SDHA","UQCRC1","COX5A","ATP5F1A"
]

SENESCENCE_CORE = [
    "CDKN1A","CDKN2A","CDKN1B","TP53",
    "GADD45A","GADD45B",
    "ATM","ATR","CHEK1","CHEK2",
    "SERPINE1","GLB1","IGFBP7"
]

SASP = [
    "IL6","IL8","CXCL1","CXCL2","CCL2","CCL20",
    "MMP1","MMP3","MMP9",
    "ICAM1","PTGS2","TNFAIP3",
    "TGFB1","SERPINE1","PLAUR","IL1B"
]

PROLIFERATION = [
    "MKI67","TOP2A","PCNA",
    "MCM2","MCM3","MCM4","MCM5",
    "TYMS","CCNB1","CCNB2","CDK1","AURKB"
]

print("Gene sets loaded.")


Gene sets loaded.


In [ ]:
import scanpy as sc
import numpy as np
from pathlib import Path
from sklearn.cluster import KMeans

GSE = "GSE162931"
BASE = Path("data")/GSE
adata = sc.read_h5ad(BASE/f"{GSE}_merged.h5ad")

# metadata
meta = {
    "GSM4967236": dict(line="GBM131", condition="naive", timepoint="0d"),
    "GSM4967237": dict(line="GBM131", condition="RT",    timepoint="2d"),
    "GSM4967239": dict(line="GBM131", condition="RT",    timepoint="5d"),
    "GSM4967241": dict(line="GBM131", condition="RT",    timepoint="3w"),
    "GSM4967242": dict(line="GBM022", condition="naive", timepoint="0d"),
    "GSM4967244": dict(line="GBM022", condition="RT",    timepoint="2d"),
    "GSM4967246": dict(line="GBM827", condition="naive", timepoint="0d"),
    "GSM4967248": dict(line="GBM827", condition="RT",    timepoint="2d"),
}
adata.obs["line"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["line"])
adata.obs["condition"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["condition"])
adata.obs["timepoint"] = adata.obs["GSM"].map(lambda x: meta[str(x)]["timepoint"])

# output
OUTDIR = Path("out_GSE162931")
OUTDIR.mkdir(exist_ok=True)
sc.settings.figdir = str(OUTDIR)
sc.settings.set_figure_params(dpi=220, fontsize=10)

# preprocess
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=10)
adata.var["mt"] = adata.var_names.str.upper().str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
adata = adata[adata.obs.pct_counts_mt < 20].copy()

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000, subset=True)
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata)

# neighbors/umap（ここはigraph不要）
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)
sc.tl.umap(adata)

# --- 代替クラスタ：KMeans（igraph不要）---
k = 10  # 8〜15くらいでOK（助成金用）
km = KMeans(n_clusters=k, random_state=0, n_init="auto")
adata.obs["kmeans"] = km.fit_predict(adata.obsm["X_pca"][:, :30]).astype(str)

def score(genes, name):
    g = [x for x in genes if x in adata.var_names]
    if len(g) < 4:
        adata.obs[name] = np.nan
        print(f"[WARN] {name}: found {len(g)} genes")
        return
    sc.tl.score_genes(adata, g, score_name=name)

# gene sets（先にあなたが実行したセルの定義を使う）
score(STEMNESS, "StemnessScore")
score(METABOLIC_SIMPLE, "MetabolicScore")
score(CIRCADIAN, "CircadianScore")
score(SENESCENCE_CORE, "SenescenceCoreScore")
score(SASP, "SASPScore")
score(PROLIFERATION, "ProliferationScore")

# plots
sc.pl.umap(adata, color=["line","condition","timepoint","kmeans"], wspace=0.4, show=False, save="_meta.png")
sc.pl.umap(adata, color=["StemnessScore","MetabolicScore","CircadianScore","SenescenceCoreScore","SASPScore","ProliferationScore"],
           wspace=0.4, show=False, save="_scores.png")

sc.pl.violin(adata,
             keys=["StemnessScore","MetabolicScore","CircadianScore","SenescenceCoreScore","SASPScore","ProliferationScore"],
             groupby="condition", rotation=45, show=False, save="_scores_by_condition.png")

sc.pl.violin(adata,
             keys=["StemnessScore","MetabolicScore","CircadianScore","SenescenceCoreScore","SASPScore","ProliferationScore"],
             groupby="timepoint", rotation=45, show=False, save="_scores_by_timepoint.png")

adata.write(OUTDIR/"GSE162931_processed_scores_noigraph.h5ad")
print("DONE. Figures in:", OUTDIR)


/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1806: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  

[WARN] MetabolicScore: found 1 genes
[WARN] CircadianScore: found 3 genes
DONE. Figures in: out_GSE162931
